# Connecting to NAPS SQL Server with JayDeBeApi

**GOAL:** Query SQL Server using Single Sign-On (SSO) and return results as a pandas dataframe

To get started, need 2 things:

1. Open-source jtds drivers from [here](https://sourceforge.net/projects/jtds/files/jtds/).
2. Java runtime environment (JRE).  **NOTE:** If you also need to query mainframe DB2, it is recommended that you use the same JRE used to connect to mainframe DB2

Then review this StackOverflow [example](https://stackoverflow.com/questions/55376100/create-a-jtds-connection-string-in-python) for SQL Server connection string details.

Review this StackOverflow [example](https://stackoverflow.com/questions/13267764/i-o-error-sso-failed-native-sspi-library-not-loaded) to determine where to save the "ntlmauth.dll" file used for SSO ("Single Sign On" authentication).  Looks like you need to save it in the `java/bin` or `jre/bin` folder.

In [1]:
from getpass import getpass
import jaydebeapi as jdpa
import pandas as pd

In [2]:
!echo %JAVA_HOME%

C:\Users\MA17151\java8


#### Database server details

In [3]:
host = 'HAMPXWSQL028'
port = '1433'
database = 'NAPS_Reporting'

#### Query to obtain NAPS MBPNs.  Please use `WITH (NOLOCK)` after table name or table alias to prevent blocking others or jobs from accessing table concurrently.

In [4]:
sql = """
SELECT DISTINCT
    RTRIM(PDTLS.BasePartNumber) as "PART-NO",
    PDTLS.IsPartActive,
    RTRIM(PDTLS.MBPN) as MBPN,
    SUBSTRING(PLANT.PlantCode, 1, 3) as "PLANT-CD",
    -- Create part # version without space in middle
    REPLACE(PDTLS.BasePartNumber, ' ', '') as BPN_Without_Space,
    REPLACE(PDTLS.MBPN, ' ', '') as MBPN_Without_Space

FROM dbo.Stg_Part_Details_TBL PDTLS
    
    INNER JOIN dbo.Stg_Events_TBL EVENTS WITH (NOLOCK) ON
    PDTLS.ControlEvent = EVENTS.EventID
    
    INNER JOIN dbo.Stg_Plant_TBL PLANT WITH (NOLOCK) ON
    PLANT.PlantID = EVENTS.PlantID

WHERE
    PDTLS.IsPartActive = 1
    AND PLANT.PlantCode = 'MAP'
    AND PDTLS.PartEffectiveBeginDate <= getdate()
    AND PDTLS.PartEffectiveEndDate > getdate()
"""

#### Usually best practice to use `with` statement when making a database connection so that we can ensure database connections are closed

Connect, execute query, return results back as a pandas dataframe

In [5]:
with jdpa.connect(
    'net.sourceforge.jtds.jdbc.Driver',
    f'jdbc:jtds:sqlserver://{host}:{port}/{database};Trusted_Connection=yes;',
    jars=['C:/Users/SomeUser/jtds/jtds-1.3.1.jar']
) as conn:
    df_mbpn = pd.read_sql(sql, conn, index_col=None)

Number of (rows, columns):

In [6]:
df_mbpn.shape

(5692, 6)

First 5 rows of data:

In [7]:
df_mbpn.head()

,PART-NO,IsPartActive,MBPN,PLANT-CD,BPN_Without_Space,MBPN_Without_Space
0,00X3ATGV A000,1,00X3ATGV A000,MAP,00X3ATGVA000,00X3ATGVA000
1,00X3ATGV A100,1,00X3ATGV A100,MAP,00X3ATGVA100,00X3ATGVA100
2,00X3ATGV A200,1,00X3ATGV A200,MAP,00X3ATGVA200,00X3ATGVA200
3,00X3ATGV A300,1,00X3ATGV A300,MAP,00X3ATGVA300,00X3ATGVA300
4,00X3ATGV A400,1,00X3ATGV A400,MAP,00X3ATGVA400,00X3ATGVA400
